In [1]:
# 安装需要的项目和包
!git clone https://github.com/huggingface/transformers.git
!pip install ./transformers
!pip install sentencepiece


Cloning into 'transformers'...
remote: Enumerating objects: 239209, done.
remote: Counting objects: 100% (303/303), done.
remote: Compressing objects: 100% (195/195), done.
remote: Total 239209 (delta 140), reused 181 (delta 94), pack-reused 238906 (from 1)
Receiving objects: 100% (239209/239209), 252.34 MiB | 11.38 MiB/s, done.
Resolving deltas: 100% (175023/175023), done.
Processing ./transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 77.1 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.47.0.dev0-py3-none-any.whl size=10040589 sha256=a2cfa791a2510bb09b9731a3e572710dd30b5d70a05e0e49a918d27e81661080
  Stored in directory: /tmp/pip-ephem-wheel-cache-h43282q5/wheels/7c/35/80/e946b22a081210c6642e607ed65b2a5b9a4d9259695ee2caf5
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing in

In [2]:
# 加载模型
# 如果加载ClueAI/PromptCLUE-base-v1-5, 请替换为ClueAI/PromptCLUE-base-v1-5
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained("ClueAI/PromptCLUE-base")
model = T5ForConditionalGeneration.from_pretrained("ClueAI/PromptCLUE-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/742k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [3]:
# 使用
import torch
from transformers import AutoTokenizer
print(torch.cuda.is_available())
# 修改colab笔记本设置为gpu，推理更快
device = torch.device('cuda')
model.to(device)
def preprocess(text):
  return text.replace("\n", "_")

def postprocess(text):
  return text.replace("_", "\n")

def answer(text, sample=True, top_p=0.6):
  '''sample：是否抽样。生成任务，可以设置为True;
  top_p：0-1之间，生成的内容越多样'''
  text = preprocess(text)
  encoding = tokenizer(text=[text], truncation=True, padding=True, max_length=768, return_tensors="pt").to(device)
  if not sample:
    out = model.generate(**encoding, return_dict_in_generate=True, output_scores=False, max_new_tokens=128, num_beams=4, length_penalty=0.6)
  else:
    out = model.generate(**encoding, return_dict_in_generate=True, output_scores=False, max_new_tokens=128, do_sample=True, top_p=top_p)
  out_text = tokenizer.batch_decode(out["sequences"], skip_special_tokens=True)
  return postprocess(out_text[0])
print("end...")

True
end...


In [10]:
#读取数据
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/
import json
import csv
import pandas as pd
import re
result = {}
file = open('character.json', encoding='utf-8')
count = 0
for line in file:
    line = line.strip()  # 去掉首尾空白字符
    if line:
        try:
            item_dict = json.loads(line)
            result[count]=item_dict
            count+=1
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e} - Line: {line}")

file=open('relation_mod.json', encoding='utf-8')
# line=json.loads(line)
headers = ['END_ID', 'type', 'START_ID']
with open('character2character.csv', 'w', encoding='utf-8-sig', newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(headers)
    for line in file:
        line = line.strip()
        if line:
            try:
                line=json.loads(line)
                # print(line)
                chname=line['全名/本名']
                for chId in result.keys():
                    text = result[chId]['全名/本名']
                    ch = re.sub(r'（.*?）', '', text)
                    if ch in chname:
                        startId = '#'+str(chId)
                        for sen in line["语音"]:
                            a=sen.split(":")[0]
                            b=sen.split(":")[1]
                            for endId in result.keys():
                                text2 = result[endId]['全名/本名']
                                ch2 = re.sub(r'（.*?）', '', text2)
                                if ch2 in a and ch!=ch2:
                                    input_string=f'''阅读段落，抽取{ch}和{ch2}关系：{b}
                                    选项：兄弟姐妹，友谊，师徒，竞争，认识，了解，敌对
                                    答案：'''
                                    ans=answer(input_string)
                                    if ans!='陌生人' and ans!='不认识'and ans!='不详':
                                      data = [['#'+str(endId),ans , startId]]
                                      print(data)
                                      csv_writer.writerows(data)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e} - Line: {line }")
print('character2.csv finish!')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks
[['#39', '友谊', '#16']]
[['#24', '同事', '#16']]
[['#23', '知更鸟，敌对', '#16']]
[['#18', '敌对', '#16']]
[['#50', '知道', '#45']]
[['#28', '喜欢', '#45']]
[['#51', '中*人，兄弟姐妹', '#45']]
[['#9', '知书达理', '#45']]
[['#11', '师徒', '#45']]
[['#13', '师徒', '#45']]
[['#47', '友谊', '#45']]
[['#54', '认识', '#45']]
[['#22', '敌对', '#42']]
[['#71', '了解', '#42']]
[['#28', '友谊', '#42']]
[['#27', '友谊', '#42']]
[['#6', '喜欢玩火的人，友谊', '#42']]
[['#48', '知', '#42']]
[['#90', '中，敌对', '#82']]
[['#78', '同事', '#82']]
[['#92', '了解', '#82']]
[['#89', '同事', '#82']]
[['#79', '同事', '#82']]
[['#86', '中伤，师徒', '#82']]
[['#77', '知', '#82']]
[['#33', '认识', '#3']]
[['#0', '同事', '#3']]
[['#25', '不简单', '#3']]
[['#6', '知', '#3']]
[['#22', '认识', '#3']]
[['#36', '师徒', '#3']]
[['#52', '不信任', '#3']]
[['#53', '师徒', '#3']]
[['#16', '了解', '#23']]
[['#39', '喜欢', '#23']]
[['#1', '同事', '#